In [1]:
#dependencies

#from sklearn import tree
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer

In [1]:
#Read in file
df = pd.read_csv('data/big_filtered.csv')
#drop the untitled index column
df=df.iloc[:, 1:]

NameError: name 'pd' is not defined

In [16]:
#break languages out into categorical vars
df['language']=df['language'].apply(lambda x: x.replace(',', '') if x==x else '')
cvt=CountVectorizer()
language_ary=cvt.fit_transform(df['language']).toarray()
# feature_names=[f'language_{language}' for cvt.get_feature_names()
# feature_names=[]
language_df=pd.DataFrame(language_ary, columns=[f'{language}' for language in cvt.get_feature_names()])
#language_df.head()

In [6]:
#renaming to save typing
df1=language_df

In [8]:
#now need an index on the wide categorical df
df1.reset_index(inplace=True, drop=False)


In [13]:
#renaming index column to ID_ to prevent reserved word clash

df1a = df1.rename({'index': 'ID_'}, inplace=False, axis=1)
df1a.drop('level_0', inplace=True, axis=1)


In [56]:
df2.shape

(1419775, 3)

In [20]:
#unpivot the array
df2 = df1a.melt(id_vars='ID_', var_name='language', value_name = 'language_indicator')

In [57]:
#now get rid of all the zeroes
df3=df2[df2['language_indicator']>0]

In [58]:
#now that the above is done, don't need the indicator anymore, since each row is implicitly an indicator
df3 = df3.drop('language_indicator', axis=1)

In [31]:
#categorize languages into order

order_dict = {
"american":1
,"english":1
,"spanish":2
,"french":3
,"russian":4
,"german":5
,"italian":6
,"japanese":7
,"cantonese":8
,"chinese":8
,"mandarin":8
,"shanghainese":8
,"arabic":9
,"korean":10
}

In [59]:
s = df3['language']
t=[]
for lang in s:
   t.append(order_dict.get(lang,999)) 

In [62]:
df3['order']=t

In [67]:
#dropping language column to minimize group by dimensions
df4 = df3.drop('language', axis=1)

In [68]:
df4.head(10)

,ID_,order
3276,3276,999
6002,6002,999
7651,7651,999
7907,7907,999
9044,931,999
9289,1176,999
10672,2559,999
11568,3455,999
11605,3492,999
12970,4857,999


In [69]:

df5 = df4.groupby('ID_').min()

In [71]:
df5.head()

,order
ID_,
0,1
1,999
2,999
3,1
4,10


In [104]:
#df = df.reset_index(drop=False)
#df = df.drop('level_0', axis=1)
df = df.rename({'index': 'ID_'}, axis=1)
df.head()

,ID_,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,...,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics,intyear,budget_curr,budget_amt,budget_int,ROI_pct,roi_outlier_ind
0,0,tt6793280,The Devil's Doorway,The Devil's Doorway,2018,2/28/2019,Horror,76,"Ireland, UK",English,...,516660,48.0,48.0,45.0,2018,$,1,1,5.166590e+07,True
1,1,tt10183542,Firincinin Karisi,Firincinin Karisi,2019,9/27/2019,Comedy,91,Turkey,Turkish,...,471182,NaN,NaN,1.0,2019,$,2,2,2.355900e+07,True
2,2,tt1783232,Fetih 1453,Fetih 1453,2012,2/16/2012,"Action, Drama, History",162,Turkey,Turkish,...,35797045,NaN,154.0,21.0,2012,$,18,18,1.988724e+08,True
3,3,tt0122143,The Last Broadcast,The Last Broadcast,1998,10/23/1998,"Horror, Mystery",86,USA,English,...,12097,NaN,171.0,46.0,1998,$,900,900,1.244111e+03,True
4,4,tt1127855,Ri-teon,Ri-teon,2007,8/8/2007,"Drama, Thriller",113,South Korea,Korean,...,4569780,NaN,1.0,11.0,2007,$,1000,1000,4.568780e+05,True


In [74]:
#Bringing back in language values now that we have at most 1 lang/movie

lang_dict = {
1:"english",
2:"spanish",
3:"french",
4:"russian",
5:"german",
6:"italian",
7:"japanese",
8:"chinese",
9:"arabic",
10:"korean",
999:"other"
}

In [78]:
r = df5['order']
s = []
for lang in r:
    s.append(lang_dict.get(lang)) 

In [80]:
s[:5]

['english', 'other', 'other', 'english', 'korean']

In [81]:
df5['lang'] = s

In [105]:
df6 = df.merge(df5, how='left', on='ID_', sort=True)

In [106]:
df6.head()

,ID_,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,...,reviews_from_users,reviews_from_critics,intyear,budget_curr,budget_amt,budget_int,ROI_pct,roi_outlier_ind,order,lang
0,0,tt6793280,The Devil's Doorway,The Devil's Doorway,2018,2/28/2019,Horror,76,"Ireland, UK",English,...,48.0,45.0,2018,$,1,1,5.166590e+07,True,1.0,english
1,1,tt10183542,Firincinin Karisi,Firincinin Karisi,2019,9/27/2019,Comedy,91,Turkey,Turkish,...,NaN,1.0,2019,$,2,2,2.355900e+07,True,999.0,other
2,2,tt1783232,Fetih 1453,Fetih 1453,2012,2/16/2012,"Action, Drama, History",162,Turkey,Turkish,...,154.0,21.0,2012,$,18,18,1.988724e+08,True,999.0,other
3,3,tt0122143,The Last Broadcast,The Last Broadcast,1998,10/23/1998,"Horror, Mystery",86,USA,English,...,171.0,46.0,1998,$,900,900,1.244111e+03,True,1.0,english
4,4,tt1127855,Ri-teon,Ri-teon,2007,8/8/2007,"Drama, Thriller",113,South Korea,Korean,...,1.0,11.0,2007,$,1000,1000,4.568780e+05,True,10.0,korean


In [107]:
# save work as csv 
df6.to_csv('lang_reduced.csv')

In [110]:
df6['lang']=df6['lang'].apply(lambda x: x.replace(',', '') if x==x else '')
cvt=CountVectorizer()
language_ary=cvt.fit_transform(df6['lang']).toarray()
# feature_names=[f'language_{language}' for cvt.get_feature_names()
# feature_names=[]
language_df=pd.DataFrame(language_ary, columns=[f'{language}' for language in cvt.get_feature_names()])
language_df.head(10)

,arabic,chinese,english,french,german,italian,japanese,korean,other,russian,spanish
0,0,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,1,0,0
3,0,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,0
5,0,0,1,0,0,0,0,0,0,0,0
6,0,0,1,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0
8,0,0,1,0,0,0,0,0,0,0,0
9,0,0,1,0,0,0,0,0,0,0,0


In [111]:
language_df.shape

(8113, 11)

In [112]:
language_df.to_csv('language_ary.csv')